# **ТЕОРІЯ ЙМОВІРНОСТІ ТА МАТЕМАТИЧНА СТАТИСТИКА**
## *Лабораторна робота №2* - Перевірка статистичних гіпотез

### **Завдання**
##### *На основі змодельованих даних реалізувати алгоритми перевірки критеріїв Колмогорова та Вілкоксона*

In [1]:
import numpy as np
from math import factorial
from scipy.stats import binom

#### Критерій знаків

##### *Дані з прикладу для перевірки*

In [2]:
n = 100
data1 = [158,154,145,143,158,152,144,150,160,145,144,148,154,152,141,144,144,144,149,148,145,158,
         147,143,157,150,140,160,148,147,146,150,142,144,143,149,153,159,157,156,144,147,153,143,
         149,145,150,146,153,157,145,144,140,151,153,154,160,145,143,153,149,148,144,143,154,154,
         152,158,150,142,143,143,156,151,144,144,144,153,145,140,155,157,149,141,150,150,154,153,
         156,142,154,157,159,156,150,157,142,157,144,154]
data2 = [136,142,135,155,150,140,150,153,145,154,138,136,154,138,155,155,144,141,142,153,152,150,
         137,141,135,143,149,138,136,145,145,152,152,140,137,144,155,142,154,136,147,143,137,138,
         146,155,152,137,144,135,135,136,136,147,140,141,143,135,139,151,148,135,153,147,144,154,
         138,141,135,149,136,154,144,140,148,144,149,147,135,141,147,150,139,141,145,136,147,139,
         147,153,149,136,147,153,139,153,141,155,137,147]

In [3]:
n = 100
data1 = np.random.rand(100) * 20 + 100
data2 = np.random.rand(100) * 20 + 100
print(data1)
print(data2)

[100.73141117 100.61859875 101.68537182 103.75050312 102.55865511
 101.67703135 118.8641453  112.86323889 103.54663639 101.760123
 117.95866412 117.18232007 102.45560759 106.84758907 107.31879272
 105.76459298 115.56400424 107.79624981 110.96944528 108.77113363
 118.13175827 115.56701017 106.46997245 116.85722134 104.37203472
 105.39609535 112.93558448 105.94082177 103.77216316 110.16130979
 111.99940141 104.8750298  109.65677136 111.2520537  110.6614516
 105.92397522 106.84589746 115.48093197 100.34622661 107.44713444
 103.749679   111.50412242 111.86575989 119.50090749 118.49690654
 103.98659889 103.53760003 110.17478472 117.63808669 101.17287904
 110.94467938 114.21917802 117.45492168 109.03900864 108.11677355
 108.43652648 106.1536029  118.36796265 119.99875251 100.3741965
 101.94186072 113.53280127 103.20557437 114.28313124 118.39865222
 107.7892745  108.25369191 104.24959336 113.13881646 103.04028623
 104.15506512 119.08815328 113.66963748 112.16696563 106.86866261
 108.80683212 

In [4]:
signes = ['+' if data1[i]-data2[i] > 0 else '-' for i in range(n)]
print(signes)
positives = signes.count('+')
print(positives)

['-', '-', '-', '-', '+', '-', '+', '+', '-', '-', '+', '+', '-', '-', '-', '-', '+', '-', '-', '-', '+', '-', '+', '+', '-', '-', '+', '-', '-', '+', '+', '-', '-', '-', '-', '-', '-', '+', '-', '+', '+', '-', '+', '+', '+', '-', '-', '+', '-', '-', '+', '+', '-', '-', '-', '-', '-', '+', '+', '-', '-', '+', '-', '+', '+', '-', '-', '-', '+', '-', '-', '+', '+', '+', '-', '+', '+', '+', '+', '-', '-', '-', '-', '+', '+', '-', '-', '-', '+', '-', '+', '-', '+', '+', '+', '+', '+', '+', '-', '-']
44


In [5]:
alpha = 0.01
p = 0
i = n-1
while p <= alpha:
    p = sum([factorial(n) / (factorial(j) * factorial(n-j) * 2**n) for j in range(i, n)])
    i -= 1
critical = i
print("Нижнє критичне значення:", n-critical)
print("Верхнє критичне значення:", critical)

Нижнє критичне значення: 39
Верхнє критичне значення: 61


In [6]:
print(binom.ppf(alpha, 100, 0.5))

38.0


In [7]:
if n - critical <= positives <= critical:
    print("Контролери отримують однакові результати")
else:
    print("Контролери отримують не однакові результати")

Контролери отримують однакові результати


#### Критерій Колмагорова

#### Критерій Вілкоксона

n - коротша вибірка, m - довша

In [8]:
from scipy.stats import norm
from math import sqrt

def vilk_stat(shorter_sample, longer_sample):
    var_series = np.sort(np.array([(i, 's', 0) for i in shorter_sample] + [(i, 'l', 0) for i in longer_sample]), axis=0)
    unique, counts = np.unique(var_series[:,0], return_counts=True)
    ranks = []
    for i in range(len(counts)):
        elems = counts[i]
        start_rank = sum(counts[:i]) + 1
        end_rank = sum(counts[:i+1])
        for j in range(elems):
            ranks.append(sum([k for k in range(start_rank, end_rank+1)]) / elems)
    for i, r in enumerate(ranks):
        var_series[i,2] = r
    shorter = var_series[var_series[:, 1] == 's'][:,2]
    return np.sum(shorter.astype('float64'))

def crit_stat(n, m):
    z = norm.ppf(0.01)
    return 0.5 * n * (n + m + 1) + z * sqrt(1/12 * n * m * (n + m + 1))
    
n, m = len(data1), len(data2)
statistics = vilk_stat(data1, data2)
critical = crit_stat(n, m)
print('W =', statistics)
print('Wcrit = [', critical, n * (n + m + 1) - critical, ']')

W = 15050.0
Wcrit = [ 9097.901099044011 11002.098900955989 ]


In [9]:
if critical <= statistics <= n * (n + m + 1) - critical:
    print("Гіпотеза H0 справджується")
else:
    print("Гіпотеза H0 не справджується")

Гіпотеза H0 не справджується
